In [1]:
import pandas as pd

# Load final merged dataset
df = pd.read_csv("final_food_delivery_dataset.csv")

# Convert order_date (DD-MM-YYYY format)
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True)

# Clean membership values
df["membership"] = df["membership"].str.strip().str.title()

# Remove duplicate restaurant name column if present
df = df.drop(columns=["restaurant_name_x"], errors="ignore")
df = df.rename(columns={"restaurant_name_y": "restaurant_name"})

print("Dataset loaded:", df.shape)


Dataset loaded: (10000, 11)


In [3]:
# Q1: City with highest revenue from Gold members
gold_city_revenue = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)
gold_city_revenue


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [5]:
# Q2: Cuisine with highest average order value
df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [7]:
# Q3: Users with total spend > ₹1000
user_spend = df.groupby("user_id")["total_amount"].sum()
(user_spend > 1000).sum()


2544

In [9]:
# Q4: Rating range with highest revenue
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

df["rating_range"] = pd.cut(df["rating"], bins=bins, labels=labels)

df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


C:\Users\praju\AppData\Local\Temp\ipykernel_20200\753691683.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


rating_range
4.6–5.0    2197030.75
4.1–4.5    1960326.26
3.0–3.5    1881754.57
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [11]:
# Q5: City with highest AOV among Gold members
df[df["membership"] == "Gold"].groupby("city")["total_amount"].mean().sort_values(ascending=False)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [13]:
# Q6: Cuisine with fewer restaurants but strong revenue
restaurant_count = df.groupby("cuisine")["restaurant_name"].nunique()
revenue_by_cuisine = df.groupby("cuisine")["total_amount"].sum()

pd.DataFrame({
    "restaurants": restaurant_count,
    "revenue": revenue_by_cuisine
}).sort_values("restaurants")


,restaurants,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [15]:
# Q7: Percentage of orders by Gold members
gold_orders = df[df["membership"] == "Gold"].shape[0]
total_orders = df.shape[0]

round((gold_orders / total_orders) * 100)


50

In [17]:
# Q8: Restaurant with highest AOV and less than 20 orders
restaurant_stats = df.groupby("restaurant_name").agg(
    avg_order_value=("total_amount", "mean"),
    total_orders=("order_id", "count")
)

restaurant_stats[restaurant_stats["total_orders"] < 20].sort_values(
    "avg_order_value", ascending=False
).head()


,avg_order_value,total_orders
restaurant_name,,
Restaurant_294,1040.222308,13
Restaurant_262,1029.473333,18
Restaurant_77,1029.180833,12
Restaurant_193,1026.306667,15
Restaurant_7,1002.140625,16


In [19]:
# Q9: Highest revenue combination
df.groupby(["membership", "cuisine"])["total_amount"].sum().sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [21]:
# Q10: Quarter with highest revenue
df["quarter"] = df["order_date"].dt.to_period("Q")
df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [23]:
# 1. Total orders by Gold members
df[df["membership"] == "Gold"].shape[0]


4987

In [25]:
# 2. Total revenue from Hyderabad (rounded)
round(df[df["city"] == "Hyderabad"]["total_amount"].sum())


1889367

In [27]:
# 3. Distinct users
df["user_id"].nunique()


2883

In [29]:
# 4. Average order value for Gold members
round(df[df["membership"] == "Gold"]["total_amount"].mean(), 2)


797.15

In [31]:
# 5. Orders with rating >= 4.5
df[df["rating"] >= 4.5].shape[0]


3374

In [33]:
# 6. Orders in top Gold revenue city
top_gold_city = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

df[
    (df["membership"] == "Gold") &
    (df["city"] == top_gold_city)
].shape[0]


1337